In [2]:
import numpy as np
import pandas as pd
import math
import time
from xclib.data import data_utils

In [15]:
datapath = './../virus/ass3_parta_data/'
num_classes = 2

In [16]:
def read_datafile(filepath):
    temp = data_utils.read_sparse_file(filepath, force_header=True)
    return temp

In [17]:
class Node:
    
    def __init__(self, split_on, prediction, depth):
        self.split_on = split_on
        self.threshold = 0.0
        self.left = None
        self.right = None
        self.depth = depth
        self.prediction = prediction
    
    def is_leaf(self, y):
        s = np.sum(y)
        if s==0 or s==len(y):
            return True
        return False
    
    def entropy(self, y):
        total = len(y)
        num1 = np.sum(y)
        num0 = total - num1
        if(num0==0 or num1==0):
            return 0.0
        return (num0/total)*math.log(total/num0) + (num1/total)*math.log(total/num1)

    def choose_best_attr2(self, X_tgt, y_tgt):
        num_features = len(X_tgt[0])
        entropy_curr = self.entropy(y_tgt)
        # print("Entropy curr = ", entropy_curr)
        total = len(y_tgt)
        best_diff = 0
        best_attr = 0
        best_thres = 0
        # t_iter_start = time.time()
        for i in range(num_features):
            # t1 = time.time()
            # compute threshold
            thres = np.median(X_tgt[:, i])
            
            # t2 = time.time()
            # print("computing threshold: ",t2-t1)
            # Split wrt threshold
            filt_left = X_tgt[:,i]<=thres
            filt_right = ~filt_left
            # X_left = X_tgt[filt_left]
            # X_right = X_tgt[filt_right]
            y_left = y_tgt[filt_left]
            y_right = y_tgt[filt_right]
            if len(y_left)==0 or len(y_right)==0:
                continue
            
            # t1=time.time()
            # print("split wrt threshold: ", t1-t2)
            # Calculate left and right entropies
            totall = len(y_left)
            num1l = np.sum(y_left)
            num0l = totall - num1l
            if(num0l==0 or num1l==0):
                entropy_left = 0.0
            else:
                entropy_left = (num0l/totall)*math.log(totall/num0l) + (num1l/totall)*math.log(totall/num1l)
            totalr = len(y_right)
            num1r = np.sum(y_right)
            num0r = totalr - num1r
            if(num0r==0 or num1r==0):
                entropy_right = 0.0
            else:
                entropy_right = (num0r/totalr)*math.log(totalr/num0r) + (num1r/totalr)*math.log(totalr/num1r)
            diff = entropy_curr - (entropy_left*(totall/total) + entropy_right*(totalr/total))

            # print("i = ", i)
            # print("Entropy left = ", entropy_left)
            # print("Entropy right = ", entropy_right)
            # print("Threshold = ", thres)
            if (diff >= best_diff):
                best_diff = diff
                best_attr = i
                best_thres = thres
            # t2 = time.time()
            # print("Calculate entropies and compare: ", t2-t1)

        if(best_diff <= 0):
            return [-1, -1]
        # t_iter_end = time.time()
        # print("t_iter: ", t_iter_end-t_iter_start)
        return [best_attr, best_thres]

In [18]:
class MyDecisionTreeClassifier:

    def __init__(self, max_depth=-1):
        self.root = None
        self.max_depth = max_depth
        self.num_leaves = 0
        self.num_nodes = 0
        self.tree_depth = 0

    def fit(self, X, y):
        self.root = self.grow_tree(X, y)

    def grow_tree(self, X, y, depth=0):
        self.tree_depth = max(self.tree_depth, depth)

        # print("Depth = ", depth)
        # print("Size = ", len(y))

        num_tgt = len(y)
        if(np.sum(y) > num_tgt/2):
            prediction = 1.0
        else:
            prediction = 0.0
        
        self.num_nodes += 1
        node = Node(split_on=-1, prediction=prediction, depth=depth)

        if(node.is_leaf(y) or (node.depth == self.max_depth and self.max_depth!=-1)):
            self.num_leaves += 1
            # print("Split on: leaf\n")
            return node
        
        split_on, thres = node.choose_best_attr2(X, y)
        ind_left = X[:,split_on] <= thres
        ind_right = X[:,split_on] > thres
        node.split_on = split_on
        node.threshold = thres
        # print("Split on: {}\n".format(split_on))

        if split_on == -1:
            self.num_leaves += 1
            # print("Split on: leaf (no info gain)\n")
            return node

        node.left = self.grow_tree(X[ind_left], y[ind_left], depth=depth+1)
        node.right = self.grow_tree(X[ind_right], y[ind_right], depth=depth+1)

        return node
    
    def predict_sample(self, x):
        node = self.root
        pred = -1.0
        while node!=None:
            pred = node.prediction
            split_attr = node.split_on
            split_val = node.threshold
            if x[split_attr] <= split_val:
                node = node.left
            else:
                node = node.right
        return pred
    
    def predict(self, X_test):
        y_preds = np.array([self.predict_sample(x) for x in X_test])
        return y_preds

In [25]:
train_data = read_datafile(datapath + 'train_x.txt')
X_train = np.array(train_data.toarray(), dtype=int)
y_train = np.genfromtxt(datapath + 'train_y.txt', dtype=int).reshape((-1,1))
D_train = np.append(X_train, y_train, axis=1)
print(X_train.shape)
print(y_train.shape)

(64713, 482)
(64713, 1)


In [24]:
train_data.toarray()

array([[2., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [9., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
# np.savetxt('X_train2.csv', X_train, fmt='%d')
# np.savetxt('y_train.csv', y_train)

In [26]:
# X_train = X_train[:10]
# y_train = y_train[:10]

# Train a Decision tree classifier
start = time.time()
clf = MyDecisionTreeClassifier(max_depth=-1)
clf.fit(X_train, y_train)

end = time.time()
print("Tree depth = {}".format(clf.tree_depth))
print("Total number of nodes = {}".format(clf.num_nodes))
print("Total leaf nodes = {}".format(clf.num_leaves))
print("Time taken = {} s".format(end - start))

Tree depth = 54
Total number of nodes = 20003
Total leaf nodes = 10002
Time taken = 179.8130259513855 s


In [27]:
# Making predictions
X_test = read_datafile(datapath + 'train_x.txt').toarray()
y_preds = clf.predict(X_test)
y_test = np.genfromtxt(datapath + 'train_y.txt', dtype=float)
check = (y_preds == y_test)
accuracy = np.sum(check) * 100.0 / len(check)
print("Training accuracy = {}%".format(accuracy))
X_test = read_datafile(datapath + 'test_x.txt').toarray()
y_preds = clf.predict(X_test)
y_test = np.genfromtxt(datapath + 'test_y.txt', dtype=float)
check = (y_preds == y_test)
accuracy = np.sum(check) * 100.0 / len(check)
print("Test accuracy = {}%".format(accuracy))
X_test = read_datafile(datapath + 'valid_x.txt').toarray()
y_preds = clf.predict(X_test)
y_test = np.genfromtxt(datapath + 'valid_y.txt', dtype=float)
check = (y_preds == y_test)
accuracy = np.sum(check) * 100.0 / len(check)
print("Validation accuracy = {}%".format(accuracy))

Training accuracy = 90.44086968615271%
Test accuracy = 77.97969496082703%
Validation accuracy = 77.73039124791396%


In [13]:
# np.savetxt('X_test.csv', X_test, fmt='%d')
# np.savetxt('X_valid.csv', X_valid, fmt='%d')
# np.savetxt('y_test.csv', y_test, fmt='%d')
# np.savetxt('y_valid.csv', y_valid, fmt='%d')

In [279]:
# # Analyze number of nodes vs accuracy by varying depth
# # tree_depths = [60]
# tree_depths = np.arange(0, 56, step=1.0, dtype=float)
# accuracies = np.zeros((len(tree_depths),3))
# num_nodes_all = np.zeros(len(tree_depths))
# num_leaves_all = np.zeros(len(tree_depths))

# for i in range(len(tree_depths)):
#     start = time.time()
#     clf = MyDecisionTreeClassifier(max_depth=tree_depths[i])
#     clf.fit(X_train, y_train)

#     end = time.time()
#     print("Tree depth = {}".format(clf.tree_depth))
#     print("Total number of nodes = {}".format(clf.num_nodes))
#     print("Total leaf nodes = {}".format(clf.num_leaves))
#     print("Time taken = {} s".format(end - start))
#     num_nodes_all[i] = clf.num_nodes
#     num_leaves_all[i] = clf.num_leaves

#     test_sets = {0: {'x': datapath+'train_x.txt', 'y': datapath + 'train_y.txt'},
#                  1: {'x': datapath+'test_x.txt', 'y': datapath + 'test_y.txt'},
#                  2: {'x': datapath+'valid_x.txt', 'y': datapath + 'valid_y.txt'}}

#     for key in test_sets:
#         X_test = read_datafile(test_sets[key]['x']).toarray()
#         y_preds = clf.predict(X_test)
#         y_test = np.genfromtxt(test_sets[key]['y'], dtype=float)
#         check = (y_preds == y_test)
#         accuracy = np.sum(check) * 100.0 / len(check)
#         accuracies[i][key] = accuracy       
#         print(str(key) + " accuracy = {}%".format(accuracy))
#     print("")

Tree depth = 0
Total number of nodes = 1
Total leaf nodes = 1
Time taken = 0.0004570484161376953 s
0 accuracy = 52.77610371949994%
1 accuracy = 52.94144916786426%
2 accuracy = 52.243649174856294%

Tree depth = 1
Total number of nodes = 3
Total leaf nodes = 2
Time taken = 1.526113748550415 s
0 accuracy = 65.69777324494305%
1 accuracy = 65.64832413889017%
2 accuracy = 65.53866122751715%

Tree depth = 2
Total number of nodes = 7
Total leaf nodes = 4
Time taken = 2.811267852783203 s
0 accuracy = 66.24634926521719%
1 accuracy = 66.24171341152473%
2 accuracy = 66.15983682551456%

Tree depth = 3
Total number of nodes = 15
Total leaf nodes = 8
Time taken = 2.822997570037842 s
0 accuracy = 66.54149861697032%
1 accuracy = 66.71457048815539%
2 accuracy = 66.34526237715558%

Tree depth = 4
Total number of nodes = 31
Total leaf nodes = 16
Time taken = 2.9907660484313965 s
0 accuracy = 66.73620447205353%
1 accuracy = 66.7794724398498%
2 accuracy = 66.604858149453%

Tree depth = 5
Total number of nod

In [280]:
# np.savetxt('./../bin/Q1-aAll_nodes.txt', num_nodes_all, fmt='%d')
# np.savetxt('./../bin/Leaves.txt', num_leaves_all, fmt='%d')
# np.savetxt('./../bin/Sampled_depths.txt', tree_depths, fmt='%d')
# np.savetxt('./../bin/Accuracies.txt', accuracies, delimiter=',', fmt='%.05f')

In [281]:
from sklearn import tree
from sklearn.metrics import accuracy_score

skl_dtreeclf = tree.DecisionTreeClassifier(criterion='entropy')
skl_dtreeclf.fit(X_train, y_train)

y_skl = skl_dtreeclf.predict(X_test)
accuracy_score(y_skl, y_test)

0.7726682736881142

In [282]:
# print(root.left.left.left.left.right.right.prediction)
print(np.sum(X_train[:,216]==5))

1


In [283]:
print(X_train[:,363])
clf.root.threshold

[7. 1. 2. ... 3. 7. 3.]


0.0

In [3]:
merax = np.arange(9).reshape((3,3))
meraxcopy = merax
merafilt = merax[:,:] >= 4
merax2 = merax[merafilt]
np.bitwise_not(merafilt)
meraa = np.array([1,2,3,4], dtype=int)
merab = np.array([1,2,3,5])
np.median(np.array([[4,2,1,3]]))
merax*merafilt
~merafilt
np.median(meraa)
# meraa == merab

2.5

In [285]:
def entropy(y):
    total = len(y)
    num1 = np.sum(y)
    num0 = total - num1
    if(num0==0 or num1==0):
        return 0.0
    return (num0/total)*math.log(total/num0) + (num1/total)*math.log(total/num1)

In [286]:
def calc_threshold(attr, indices):
    target = X_train[indices][attr].reshape(-1)
    return np.median(target)

In [287]:
# Return the samples with attr value <= threshold and > threshold
def split_data(indices, attr):
    X_tgt = X_train[indices]
    y_tgt = y_train[indices]
    thres = calc_threshold(attr, indices)
    # print("threshold: ", thres)
    filt = X_tgt[:,attr] <= thres
    return [indices[filt], indices[np.bitwise_not(filt)]]

In [288]:
def predict(tree_root, x):
    node = tree_root
    while node!=None:
        pred = node.prediction
        split_attr = node.split_on
        split_val = node.threshold
        if x[split_attr] <= split_val:
            node = node.left
        else:
            node = node.right
    return pred

In [289]:
def choose_best_attr(X_tgt, y_tgt):
    # X_tgt = X_train[self.indices]
    # y_tgt = y_train[self.indices]
    D_tgt = np.append(X_tgt, y_tgt, axis=1)
    entropy_curr = entropy(y_tgt)
    print("Entropy curr = ", entropy_curr)
    total = len(y_tgt)
    best_diff = 0
    best_attr = 0
    best_thres = 0
    # compute threshold
    thresholds = np.median(X_tgt, axis=0)
    entropy_curr = entropy(y_tgt)
    filters_left = np.array([X_tgt[:,i]<=thresholds[i] for i in range(num_features)])
    filters_right = np.array([~filt for filt in filters_left])
    Ds_left = np.array([D_tgt[filt_left] for filt_left in filters_left])
    Ds_right = np.array([D_tgt[filt_right] for filt_right in filters_right])
    
    Ds_left = Ds_left[[len(D_left)>0 for D_left in Ds_left]]
    Ds_right = Ds_right[[len(D_right)>0 for D_right in Ds_right]]

    totalls = np.array([len(D_left[:,-1]) for D_left in Ds_left])
    num1ls = np.array([np.sum(D_left[:,-1]) for D_left in Ds_left])
    num0ls = totalls - num1ls
    entropies_left = np,array([(num0ls[i]/totalls[i])*math.log(totalls[i]/num0ls[i]) + (num1ls[i]/totalls[i])*math.log(totalls[i]/num1ls[i]) if num0ls[i]!=0 and num1ls[i]!=0 else 0 for i in range(num_features)])

    totalrs = np.array([len(D_right[:,-1]) for D_right in Ds_right])
    num1rs = np.array([np.sum(D_right[:,-1]) for D_right in Ds_right])
    num0rs = totalrs - num1rs
    entropies_right = np,array([(num0rs[i]/totalrs[i])*math.log(totalrs[i]/num0rs[i]) + (num1rs[i]/totalrs[i])*math.log(totalrs[i]/num1rs[i]) if num0rs[i]!=0 and num1rs[i]!=0 else 0 for i in range(num_features)])
        
    diffs = [entropy_curr]*(num_features) - (entropies_left * (totalls/total) + entropies_right * (totalrs/total))
    best_diff = np.max(diffs)
    best_attr = np.argmax(diffs)
    best_thres = thresholds(best_attr)
        # print("i = ", i)
        # print("Entropy left = ", entropy_left)
        # print("Entropy right = ", entropy_right)
        # print("Threshold = ", thres)
    # print(best_diff)
    if(best_diff <= 0):
        return [-1, -1]
    return [best_attr, best_thres]

In [290]:
def choose_best_attr3_child(i, node):
    X_tgt = X_train[node.indices]
    y_tgt = y_train[node.indices]
    # compute threshold
    thres = np.median(X_tgt[:, i])
    entropy_curr = entropy(y_tgt)
    # Split wrt threshold
    filt_left = X_tgt[:,i]<=thres
    filt_right = np.bitwise_not(filt_left)
    X_left = X_tgt[filt_left]
    X_right = X_tgt[filt_right]
    if len(X_left)==0 or len(X_right)==0:
        return -1
    # print("i = ", i)
    # print("left ind: ", left_ind)
    # print("right ind: ", right_ind)
    # print("Entropy left = ", entropy_left)
    # print("Entropy right = ", entropy_right)
    entropy_left = entropy(y_tgt[filt_left])
    entropy_right = entropy(y_tgt[filt_right])
    diff = entropy_curr - (entropy_left + entropy_right)

    return diff

def choose_best_attr3(node):
    # X_tgt = X_train[node.indices]
    # y_tgt = y_train[node.indices]

    best_diff = 0
    best_attr = 0
    attrs = np.arange(num_features)
    # print(X_tgt.shape)
    f = lambda i: choose_best_attr3_child(i, node)
    diffs = np.array(list(map(f, attrs)))
    return np.argmax(diffs)